循環神經網路（Recurrent Neural Network，RNN）是一種用於處理序列數據的神經網路。相比一般的神經網路來說，他能夠處理序列變化的數據。比如某個單詞的意思會因為上文提到的內容不同而有不同的含義，RNN就能夠很好地解決這類問題。架構如下圖

![RNN](https://adventuresinmachinelearning.com/wp-content/uploads/2017/09/Recurrent-neural-network.png)

RNN 模型用於股價預測

[李宏毅老師的投影片](https://www.youtube.com/watch?v=xCGidAeyS4M)

X 為當前狀態下的輸入， h 表示接收到的上一個節點的輸入。

Y 為當前狀態下的輸出，而 h' 為傳遞到下一個節點的輸出。

可以看得出來：h' 與 X 、h 密切相關，當我們把很多單元結合再一起的時候，就是一種 RNN 模型

### LSTM

LSTM (Long-Short Term Memory) 是一種 RNN (循環神經網路)，跟一般深度神經網路不同，多了資料的方向性(時序關聯)，RNN 會將每一個隱藏層的結果儲存在記憶單元，並且當新的資料近來的時候，會考慮記憶單元的值去計算

LSTM 與一般 RNN 的差別
[李宏毅老師的投影片](https://www.youtube.com/watch?v=xCGidAeyS4M)

簡單的說，LSTM 一共包含 4 個元件：輸入閘道、記憶單元、輸出閘道、清除閘道

- 輸入閘道：當將feature輸入時，input gate會去控制是否將這次的值輸入
- 記憶單元：將計算出的值儲存起來，以利下個階段拿出來使用
- 輸出閘道：控制是否將這次計算出來的值output
- 清除閘道：是否將Memory清掉(format)，restart的概念。

## 使用 LSTM 模型預測股價

### 加入套件，包含 Keras (TensorFlow)、scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import time
import math
import tensorflow as tf

from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

### 設定訓練常數

In [ ]:
# 訓練樣本數比例
SPLIT_RATIO = 0.5
# 感知器記憶長度
LOOK_BACK = 240

### 設定排序隨機碼，之後能夠重現訓練過程

In [ ]:
np.random.seed(5)

### 載入市場資料，在 data 資料夾裡面：
- NVDA.csv ： 輝達 > 每分鐘間隔
- SPY.csv ： 標普 500 ETF > 每日間隔

**要注意資料欄位格式不一樣**

In [ ]:
nvda_prices = read_csv('../input/quantitative-trading/NVDA.csv', index_col=None, delimiter=',')

In [ ]:
labels = nvda_prices['close'].values

In [ ]:
dataset = labels.reshape(-1, 1)

In [ ]:
print(f'資料集 {dataset[:10]} 長度 {len(dataset)}')

## 標準化資料

In [ ]:
Scaler = MinMaxScaler(feature_range=(0, 1))
dataset = Scaler.fit_transform(dataset)

In [ ]:
split_size = int(len(dataset) * SPLIT_RATIO)
test_size = len(dataset) - split_size

train_dataset = dataset[0:split_size, :]
test_dataset = dataset[split_size:len(dataset), :]

print(f'訓練資料集 {train_dataset[:10]} 長度 {len(train_dataset)}')
print(f'測試資料集 {test_dataset[:10]} 長度 {len(test_dataset)}')

### 拆分 訓練資料、測試資料

In [ ]:
def buildDataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
train_x, train_y = buildDataset(train_dataset, LOOK_BACK)
test_x, test_y = buildDataset(test_dataset, LOOK_BACK)

## 重塑訓練資料

### [樣本, 序列(時間)跨度, 特徵]

In [ ]:
train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))

## 建立 LSTM 模型

### 使用 adam 優化器、單層 250 個神經元、隨機排除 10% 權重

In [ ]:
Model = Sequential()
Model.add(LSTM(250, input_shape=(1, LOOK_BACK)))
Model.add(Dropout(0.1))
Model.add(Dense(1))
Model.compile(loss='mse', optimizer='adam')
# 調整訓練次數可以提升準度，也可以觀察到過擬合的問題
Model.fit(train_x, train_y, epochs=1000, batch_size=240, verbose=0)

## 預測資料

In [ ]:
train_predict = Model.predict(train_x)
test_predict = Model.predict(test_x)

### 反轉預測結果，方便結果比對

In [ ]:
train_predict = Scaler.inverse_transform(train_predict)
train_y = Scaler.inverse_transform([train_y])
test_predict = Scaler.inverse_transform(test_predict)
test_y = Scaler.inverse_transform([test_y])

### 計算預測誤差 Root Mean Squared Error

In [ ]:
train_score = math.sqrt(mean_squared_error(train_y[0], train_predict[:,0]))
print(f'訓練誤差分數(RMSE)：{np.round(train_score, 3)}')

test_score = math.sqrt(mean_squared_error(test_y[0], test_predict[:,0]))
print(f'測試誤差分數(RMSE)：{np.round(test_score, 3)}')

## 整理繪圖資料

In [ ]:
train_predict_plot = np.empty_like(dataset)
train_predict_plot[:, :] = np.nan
train_predict_plot[LOOK_BACK:len(train_predict) + LOOK_BACK, :] = train_predict

In [ ]:
test_predict_plot = np.empty_like(dataset)
test_predict_plot[:, :] = np.nan
test_predict_plot[len(train_predict) + (LOOK_BACK * 2) + 1:len(dataset)-1, :] = test_predict

In [ ]:
plt.figure(figsize=(20, 7))
plt.plot(Scaler.inverse_transform(dataset))

plt.plot(train_predict_plot)
plt.plot(test_predict_plot)
plt.show()

## 匯出 CSV 檔案

In [ ]:
test_prices=Scaler.inverse_transform(dataset[test_size+LOOK_BACK:])

result = pd.DataFrame(
    data={
        "測試價格": np.around(list(test_prices.reshape(-1)), decimals=2),
        "預測價格": np.around(list(test_predict.reshape(-1)), decimals=2)
    }
)
result.to_csv("股價預測對比.csv", sep=',', index=None)
print(result)